# Семантичне ядро з інтеграцією сервера OpenBnB MCP

Цей зошит демонструє, як використовувати Семантичне ядро разом із реальним сервером OpenBnB MCP для пошуку справжнього житла Airbnb за допомогою MCPStdioPlugin. Для доступу до LLM використовується Azure AI Foundry. Щоб налаштувати змінні середовища, ви можете скористатися [уроком із налаштування](/00-course-setup/README.md).


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Створення підключення плагіна MCP

Ми підключимося до [OpenBnB MCP сервера](https://github.com/openbnb-org/mcp-server-airbnb) за допомогою MCPStdioPlugin. Цей сервер надає функціонал пошуку Airbnb через пакет @openbnb/mcp-server-airbnb.


## Створення клієнта

У цьому прикладі ми будемо використовувати Azure AI Foundry для доступу до LLM. Переконайтеся, що ваші змінні середовища налаштовані правильно.


## Налаштування середовища

Налаштуйте параметри Azure OpenAI. Переконайтеся, що у вас встановлені наступні змінні середовища:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Розуміння інтеграції OpenBnB MCP

Цей блокнот підключається до **реального сервера OpenBnB MCP**, який забезпечує функціонал пошуку Airbnb.

### Як це працює:

1. **MCPStdioPlugin**: Використовує стандартне введення/виведення для комунікації з сервером MCP
2. **Реальний NPM пакет**: Завантажує та запускає `@openbnb/mcp-server-airbnb` через npx
3. **Живі дані**: Повертає реальні дані про нерухомість Airbnb з їхніх API
4. **Відкриття функцій**: Агент автоматично виявляє доступні функції сервера MCP

### Доступні функції:

Сервер OpenBnB MCP зазвичай надає:
- **search_listings** - Пошук нерухомості Airbnb за місцем розташування та критеріями
- **get_listing_details** - Отримання детальної інформації про конкретні об'єкти
- **check_availability** - Перевірка доступності на конкретні дати
- **get_reviews** - Отримання відгуків про об'єкти
- **get_host_info** - Отримання інформації про господарів об'єктів

### Попередні вимоги:

- **Node.js**, встановлений на вашій системі
- **Інтернет-з'єднання** для завантаження пакета сервера MCP
- **NPX**, доступний (входить до складу Node.js)

### Тестування з'єднання:

Ви можете протестувати сервер MCP вручну, запустивши:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Це завантажить і запустить сервер OpenBnB MCP, до якого Semantic Kernel потім підключається для отримання реальних даних Airbnb.


## Запуск агента з сервером OpenBnB MCP

Тепер ми запустимо AI-агента, який підключається до сервера OpenBnB MCP, щоб знайти реальні варіанти проживання Airbnb у Стокгольмі для 2 дорослих і 1 дитини. Ви можете змінити список `user_inputs`, щоб змінити критерії пошуку.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Резюме  
Вітаємо! Ви успішно створили AI-агента, який інтегрується з реальним пошуком житла за допомогою протоколу Model Context Protocol (MCP):  

Використані технології:  
Semantic Kernel - для створення інтелектуальних агентів з Azure OpenAI  
Azure AI Foundry - для можливостей LLM та завершення чатів  
MCP (Model Context Protocol) - для стандартизованої інтеграції інструментів  
OpenBnB MCP Server - для реального пошуку житла через Airbnb  
Node.js/NPX - для запуску зовнішнього MCP-сервера  

Що ви дізналися:  
Інтеграція MCP: Підключення агентів Semantic Kernel до зовнішніх MCP-серверів  
Доступ до даних у реальному часі: Пошук реальних об'єктів Airbnb через живі API  
Протокольна комунікація: Використання stdio-комунікації між агентом і MCP-сервером  
Відкриття функцій: Автоматичне виявлення доступних функцій на MCP-серверах  
Стримінг відповідей: Захоплення та логування викликів функцій у реальному часі  
Рендеринг HTML: Форматування відповідей агента зі стилізованими таблицями та інтерактивними елементами  

Наступні кроки:  
Інтегрувати додаткові MCP-сервери (погода, авіарейси, ресторани)  
Створити систему з кількома агентами, що комбінує MCP та A2A протоколи  
Розробити власні MCP-сервери для ваших джерел даних  
Реалізувати постійну пам'ять розмов між сесіями  
Розгорнути агента на Azure Functions з оркестрацією MCP-серверів  
Додати автентифікацію користувачів та можливості бронювання  

Ключові переваги архітектури MCP:  
Стандартизація: Універсальний протокол для підключення AI-агентів до зовнішніх інструментів  
Дані в реальному часі: Доступ до актуальної інформації з різних сервісів  
Розширюваність: Легка інтеграція нових джерел даних та інструментів  
Сумісність: Працює з різними AI-фреймворками та платформами агентів  
Розділення обов'язків: Чітке розмежування між логікою AI та доступом до зовнішніх даних  



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
